In [2]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,Activation, Dropout

from ple.games.flappybird import FlappyBird
from ple import PLE
import numpy as np

import math
import random
from random import randint
from collections import deque


from keras.models import load_model
from keras import optimizers

In [2]:
model = Sequential()
opti = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

model.add(Dense(800, kernel_initializer='lecun_uniform', input_shape=(8,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2, kernel_initializer='lecun_uniform'))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer=opti)

In [3]:
model = load_model("model_flappy_800_800_start.dqf")

In [8]:
gamma = .999 # discount factor
epsilon = .05 # epsilon-greddy
epochs  = 50000

n = 10

def epsilon(step):
#     if step< .5 * epochs:
#         return 1.-step * 2/epochs
    return .05
#     return 1.
update=0 
experience_replay=True
batchSize = 32# mini batch size
buffer = 1000000
replay = [] # init vector buffer
h=0 # current size of the vector buffer

In [6]:
player_y_max = 512

def normalized_state_array(state):
    n_player_y = float(state.get("player_y"))
    n_dist = float(state.get("next_pipe_dist_to_player"))
    n_top = float(state.get("next_pipe_top_y"))
    n_vel = float(state.get("player_vel"))
    
    return np.array([n_player_y, n_dist, n_top, n_vel]).reshape(1,4)
    
def c_reward(state, reward):
    player_y = float(state.get("player_y")) 
    dist = float(state.get("next_pipe_dist_to_player"))
    top = float(state.get("next_pipe_top_y"))
    bottom = float(state.get("next_pipe_bottom_y"))
    
    old_player_y = float(old_state.get("player_y")) 
    

    
    if (reward == -5):
        return -50
    elif (reward == 1):
        return 10
    else :
        result = 1
        cone_centre = 0.5 * (top + bottom)        
        if abs(player_y - cone_centre) < 10 and dist < 20: 
            result += 2
        return result
#     if player_y > cone_centre :

#         return result + 0.5 - (player_y - cone_centre) / (player_y_max - cone_centre)
#     else:
#         return result + 0.5 + (player_y - cone_centre) / cone_centre

#     result = -1
#     if(player_y > top and player_y < bottom):
#         return int( dist > 50 == 'True')
#     else:
#         return -int( dist > 50 == 'False')

def n_step_update(states, rewards, new_state, new_reward):
    
    update_state = list()
    update_value = 0  
    if (len(states) == 10):
        update_state = states.pop()   

        for i in range(n):
            update = rewards.pop()
            update_value = update_value + update * gammaR**i
            rewards.appendleft(update)
            
        rem_reward = rewards.pop()

    if (len(new_state) > 0):
        states.appendleft(new_state)
        rewards.appendleft(new_reward)
    

    return update_state, update_value



def append_last_state(states, state):
    if (len(states) == 10):
        old_state = states.pop()
    states.appendleft(state)

def append_last_action(actions, action):
    if (len(actions) == 10):
        old_action = actions.pop()
    actions.appendleft(action)

    
def n_step_update_terminal_pop(states, rewards):
    
    update_state = states.pop()
#     update_value = sum(rewards)
    update_value = 0
    for i in range(n):
        update = rewards.pop()
        update_value = update_value + update * gammaR**i
        rewards.appendleft(update)
    
    rem_reward = rewards.pop()
    return update_state, update_value


def predict_action(model, state):
    qval = model.predict(state, batch_size=batchSize)
    qval_av_action = [-9999]*2
    qval_av_action[0] = qval[0][0]
    qval_av_action[1] = qval[0][1]        
    
    return np.argmax(qval_av_action)
    

In [ ]:
game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=True)
replay = [] # init vector buffer
for i in range(epochs):
    p.init()

    while(not p.game_over()):
        state = np.array(list(game.getGameState().values())).reshape(1,8)

        qval = model.predict(state, batch_size=batchSize)
#         if (random.random() < epsilon(i)): # exploration exploitation strategy    
#             if( randint(0, 9) > 7):
#                 action = 1
#             else:
#                 action = 0
            
#         else: #choose best action from Q(s,a) values
        qval_av_action = [-9999]*2
        qval_av_action[0] = qval[0][0]
        qval_av_action[1] = qval[0][1]

        action = (np.argmax(qval_av_action))

        #Take action, observe new state S'
        #Observe reward
        reward = p.act(119*action)
        new_state = np.array(list(game.getGameState().values())).reshape(1,8)

        custom_reward = reward

        #Experience replay storage
        if (len(replay) < buffer): #if buffer not filled, add to it
            replay.append((state, action, custom_reward, new_state))
        else: #if buffer full, overwrite old values
            if (h < (buffer-1)):
                h += 1
            else:
                h = 0
            replay[h] = (state, action, custom_reward, new_state)
            state = new_state
          
        if (p.game_over()):
            print("Game ", i , " pipes = ", 5+int(game.getScore()))
            
            #randomly sample our experience replay memory
            minibatch = random.sample(replay, batchSize)
            X_train = []
            y_train = []
            for memory in minibatch:
                #Get max_Q(S',a)
                old_state, action, custom_reward, new_state = memory
                old_qval = model.predict(old_state, batch_size=1)

                newQ = model.predict(new_state, batch_size=1)
                maxQ = np.max(newQ)
                y = np.zeros((1,2))
                y[:] = old_qval[:]
                if custom_reward != -5: #non-terminal state
                    update = (1 + (gamma * maxQ))
                else: #terminal state
                    update = -1000
                y[0][action] = update

                X_train.append(old_state[0],)
                y_train.append(np.array(y).reshape(2,))

            X_train = np.array(X_train)
            y_train = np.array(y_train)

            model.fit(X_train, y_train, batch_size=batchSize, epochs=1, verbose=0)
            
            
            
        # save the model every 1000 epochs
        if (i%1000 == 0 and i > 0) or (i == epochs -1):
            save_str = "model_flappy_new"+ str(i) + ".dqf"
            model.save(save_str)
#             model.save("model_dql_flappy_dense.dqf")

Game  0  pipes =  4
Game  1  pipes =  27
Game  2  pipes =  2
Game  3  pipes =  10
Game  4  pipes =  4
Game  5  pipes =  2
Game  6  pipes =  53
Game  7  pipes =  5
Game  8  pipes =  7
Game  9  pipes =  13
Game  10  pipes =  3
Game  11  pipes =  5
Game  12  pipes =  19
Game  13  pipes =  1
Game  14  pipes =  12
Game  15  pipes =  3
Game  16  pipes =  3
Game  17  pipes =  3
Game  18  pipes =  2
Game  19  pipes =  7
Game  20  pipes =  4
Game  21  pipes =  9
Game  22  pipes =  4
Game  23  pipes =  4
Game  24  pipes =  3
Game  25  pipes =  3
Game  26  pipes =  2
Game  27  pipes =  1
Game  28  pipes =  24
Game  29  pipes =  12
Game  30  pipes =  18
Game  31  pipes =  17
Game  32  pipes =  9
Game  33  pipes =  4
Game  34  pipes =  2
Game  35  pipes =  20
Game  36  pipes =  16
Game  37  pipes =  3
Game  38  pipes =  2
Game  39  pipes =  7
Game  40  pipes =  2
Game  41  pipes =  1
Game  42  pipes =  2
Game  43  pipes =  9
Game  44  pipes =  3
Game  45  pipes =  5
Game  46  pipes =  14
Game  47  

Game  373  pipes =  1
Game  374  pipes =  8
Game  375  pipes =  1
Game  376  pipes =  4
Game  377  pipes =  6
Game  378  pipes =  8
Game  379  pipes =  8
Game  380  pipes =  2
Game  381  pipes =  2
Game  382  pipes =  7
Game  383  pipes =  10
Game  384  pipes =  1
Game  385  pipes =  14
Game  386  pipes =  1
Game  387  pipes =  7
Game  388  pipes =  2
Game  389  pipes =  2
Game  390  pipes =  1
Game  391  pipes =  2
Game  392  pipes =  8
Game  393  pipes =  8
Game  394  pipes =  4
Game  395  pipes =  9
Game  396  pipes =  1
Game  397  pipes =  1
Game  398  pipes =  17
Game  399  pipes =  8
Game  400  pipes =  2
Game  401  pipes =  1
Game  402  pipes =  2
Game  403  pipes =  14
Game  404  pipes =  11
Game  405  pipes =  2
Game  406  pipes =  2
Game  407  pipes =  7
Game  408  pipes =  5
Game  409  pipes =  2
Game  410  pipes =  11
Game  411  pipes =  3
Game  412  pipes =  11
Game  413  pipes =  1
Game  414  pipes =  6
Game  415  pipes =  7
Game  416  pipes =  5
Game  417  pipes =  1
Gam

Game  745  pipes =  5
Game  746  pipes =  2
Game  747  pipes =  1
Game  748  pipes =  1
Game  749  pipes =  1
Game  750  pipes =  1
Game  751  pipes =  1
Game  752  pipes =  1
Game  753  pipes =  4
Game  754  pipes =  2
Game  755  pipes =  1
Game  756  pipes =  1
Game  757  pipes =  4
Game  758  pipes =  3
Game  759  pipes =  8
Game  760  pipes =  2
Game  761  pipes =  1
Game  762  pipes =  1
Game  763  pipes =  2
Game  764  pipes =  6
Game  765  pipes =  3
Game  766  pipes =  1
Game  767  pipes =  1
Game  768  pipes =  7
Game  769  pipes =  2
Game  770  pipes =  1
Game  771  pipes =  2
Game  772  pipes =  5
Game  773  pipes =  1
Game  774  pipes =  2
Game  775  pipes =  2
Game  776  pipes =  2
Game  777  pipes =  2
Game  778  pipes =  1
Game  779  pipes =  4
Game  780  pipes =  1
Game  781  pipes =  1
Game  782  pipes =  3
Game  783  pipes =  1
Game  784  pipes =  1
Game  785  pipes =  1
Game  786  pipes =  4
Game  787  pipes =  1
Game  788  pipes =  3
Game  789  pipes =  1
Game  790 

Game  1113  pipes =  1
Game  1114  pipes =  3
Game  1115  pipes =  1
Game  1116  pipes =  1
Game  1117  pipes =  2
Game  1118  pipes =  1
Game  1119  pipes =  1
Game  1120  pipes =  1
Game  1121  pipes =  2
Game  1122  pipes =  2
Game  1123  pipes =  1
Game  1124  pipes =  5
Game  1125  pipes =  1
Game  1126  pipes =  2
Game  1127  pipes =  1
Game  1128  pipes =  5
Game  1129  pipes =  1
Game  1130  pipes =  1
Game  1131  pipes =  2
Game  1132  pipes =  1
Game  1133  pipes =  1
Game  1134  pipes =  1
Game  1135  pipes =  1
Game  1136  pipes =  1
Game  1137  pipes =  1
Game  1138  pipes =  3
Game  1139  pipes =  2
Game  1140  pipes =  1
Game  1141  pipes =  4
Game  1142  pipes =  2
Game  1143  pipes =  1
Game  1144  pipes =  2
Game  1145  pipes =  1
Game  1146  pipes =  1
Game  1147  pipes =  1
Game  1148  pipes =  5
Game  1149  pipes =  2
Game  1150  pipes =  1
Game  1151  pipes =  1
Game  1152  pipes =  2
Game  1153  pipes =  2
Game  1154  pipes =  3
Game  1155  pipes =  6
Game  1156 

Game  1470  pipes =  1
Game  1471  pipes =  1
Game  1472  pipes =  5
Game  1473  pipes =  2
Game  1474  pipes =  6
Game  1475  pipes =  1
Game  1476  pipes =  3
Game  1477  pipes =  4
Game  1478  pipes =  2
Game  1479  pipes =  6
Game  1480  pipes =  9
Game  1481  pipes =  1
Game  1482  pipes =  2
Game  1483  pipes =  3
Game  1484  pipes =  1
Game  1485  pipes =  2
Game  1486  pipes =  2
Game  1487  pipes =  2
Game  1488  pipes =  3
Game  1489  pipes =  1
Game  1490  pipes =  1
Game  1491  pipes =  1
Game  1492  pipes =  1
Game  1493  pipes =  6
Game  1494  pipes =  3
Game  1495  pipes =  3
Game  1496  pipes =  1
Game  1497  pipes =  2
Game  1498  pipes =  4
Game  1499  pipes =  1
Game  1500  pipes =  5
Game  1501  pipes =  8
Game  1502  pipes =  6
Game  1503  pipes =  2
Game  1504  pipes =  1
Game  1505  pipes =  2
Game  1506  pipes =  7
Game  1507  pipes =  3
Game  1508  pipes =  2
Game  1509  pipes =  1
Game  1510  pipes =  4
Game  1511  pipes =  5
Game  1512  pipes =  7
Game  1513 

Game  1825  pipes =  11
Game  1826  pipes =  19
Game  1827  pipes =  1
Game  1828  pipes =  11
Game  1829  pipes =  0
Game  1830  pipes =  19
Game  1831  pipes =  3
Game  1832  pipes =  14
Game  1833  pipes =  4
Game  1834  pipes =  2
Game  1835  pipes =  3
Game  1836  pipes =  2
Game  1837  pipes =  2
Game  1838  pipes =  4
Game  1839  pipes =  7
Game  1840  pipes =  9
Game  1841  pipes =  3
Game  1842  pipes =  19
Game  1843  pipes =  14
Game  1844  pipes =  6
Game  1845  pipes =  2
Game  1846  pipes =  17
Game  1847  pipes =  8
Game  1848  pipes =  14
Game  1849  pipes =  2
Game  1850  pipes =  5
Game  1851  pipes =  8
Game  1852  pipes =  28
Game  1853  pipes =  4
Game  1854  pipes =  7
Game  1855  pipes =  2
Game  1856  pipes =  7
Game  1857  pipes =  4
Game  1858  pipes =  11
Game  1859  pipes =  11
Game  1860  pipes =  5
Game  1861  pipes =  4
Game  1862  pipes =  5
Game  1863  pipes =  2
Game  1864  pipes =  9
Game  1865  pipes =  0
Game  1866  pipes =  4
Game  1867  pipes =  2

Game  2177  pipes =  3
Game  2178  pipes =  8
Game  2179  pipes =  2
Game  2180  pipes =  10
Game  2181  pipes =  5
Game  2182  pipes =  7
Game  2183  pipes =  7
Game  2184  pipes =  35
Game  2185  pipes =  6
Game  2186  pipes =  4
Game  2187  pipes =  16
Game  2188  pipes =  12
Game  2189  pipes =  2
Game  2190  pipes =  16
Game  2191  pipes =  2
Game  2192  pipes =  29
Game  2193  pipes =  4
Game  2194  pipes =  2
Game  2195  pipes =  11
Game  2196  pipes =  16
Game  2197  pipes =  4
Game  2198  pipes =  2
Game  2199  pipes =  12
Game  2200  pipes =  2
Game  2201  pipes =  1
Game  2202  pipes =  15
Game  2203  pipes =  2
Game  2204  pipes =  13
Game  2205  pipes =  6
Game  2206  pipes =  9
Game  2207  pipes =  3
Game  2208  pipes =  7
Game  2209  pipes =  15
Game  2210  pipes =  41
Game  2211  pipes =  21
Game  2212  pipes =  9
Game  2213  pipes =  0
Game  2214  pipes =  8
Game  2215  pipes =  4
Game  2216  pipes =  24
Game  2217  pipes =  31
Game  2218  pipes =  12
Game  2219  pipes

In [30]:
model = load_model("model_flappy_82000.dqf")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 800)               4000      
_________________________________________________________________
activation_1 (Activation)    (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1602      
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 5,602
Trainable params: 5,602
Non-trainable params: 0
_________________________________________________________________


In [18]:
game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=True)

for i in range(epochs):
    p.init()
    
#     print("ratio up/down = ", up_count/(up_count + down_count))
    while(not p.game_over()):
        state = np.array(list(game.getGameState().values())).reshape(1,8)
        
        qval = model.predict(state, batch_size=batchSize)
            
        #choose best action from Q(s,a) values
        qval_av_action = [-9999]*2
        qval_av_action[0] = qval[0][0]
        qval_av_action[1] = qval[0][1]
        
        action = (np.argmax(qval_av_action))

        reward = p.act(119*action)

        if (p.game_over()):
            print("Game ", i , " pipes = ", 5+game.getScore())

Game  0  pipes =  3.0
Game  1  pipes =  8.0
Game  2  pipes =  0.0
Game  3  pipes =  4.0
Game  4  pipes =  4.0
Game  5  pipes =  4.0
Game  6  pipes =  2.0
Game  7  pipes =  4.0
Game  8  pipes =  2.0
Game  9  pipes =  2.0
Game  10  pipes =  1.0
Game  11  pipes =  2.0
Game  12  pipes =  6.0
Game  13  pipes =  6.0
Game  14  pipes =  2.0
Game  15  pipes =  8.0
Game  16  pipes =  4.0
Game  17  pipes =  2.0
Game  18  pipes =  13.0
Game  19  pipes =  2.0
Game  20  pipes =  9.0
Game  21  pipes =  3.0
Game  22  pipes =  3.0
Game  23  pipes =  4.0
Game  24  pipes =  21.0
Game  25  pipes =  2.0
Game  26  pipes =  8.0
Game  27  pipes =  10.0
Game  28  pipes =  15.0
Game  29  pipes =  3.0
Game  30  pipes =  1.0
Game  31  pipes =  7.0
Game  32  pipes =  8.0
Game  33  pipes =  12.0
Game  34  pipes =  2.0
Game  35  pipes =  4.0
Game  36  pipes =  20.0
Game  37  pipes =  22.0
Game  38  pipes =  2.0
Game  39  pipes =  8.0
Game  40  pipes =  1.0
Game  41  pipes =  4.0
Game  42  pipes =  8.0
Game  43  pipe

SystemExit: 

C:\Users\npinc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
n_width = math.floor(player_y_max/5)
n_dist = math.floor(3*dist_max)+1

n_near_width = 200
n_near_dist = 100

n_far_width =  math.floor(player_y_max/20) +1
n_far_dist = math.floor(3*dist_max/20)+1

n_width = n_near_width +n_far_width
n_dist = n_near_dist +n_far_dist

# q_val_array = np.zeros((n_width, n_width, n_width, n_dist, 2))

q_val_near = np.zeros((n_near_width, n_near_width, n_near_dist, 2))
q_val_far = np.zeros((n_far_width, n_far_width, n_far_width, n_far_dist, 2))

print(n_far_dist)

epochs = 1000
game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=False, display_screen=True)

p.init()

for i in range(epochs):
    p.init()
    while(not p.game_over()):
        state = game.getGameState()
        
        player_y = int(state.get("player_y"))
        top = int(state.get("next_pipe_top_y"))
        bottom = int(state.get("next_pipe_bottom_y"))
        dist = int(state.get("next_pipe_dist_to_player"))

        if (random.random() < epsilon(i)): # exploration exploitation strategy    
            action = int(bool(random.getrandbits(1)) == True)
            print("random action = ", action)
        else: #choose best action from Q(s,a) values
            if (player_y - top < n_near_width/2 and player_y - bottom < n_near_width and dist < n_near_dist):
                qval = q_val_near[player_y - top + math.floor(n_near_width/2)][player_y - bottom + math.floor(n_near_width/2)][dist][:]
            else :
                player_y = math.floor(int(state.get("player_y"))/20) 
                top = math.floor(int(state.get("next_pipe_top_y"))/20)
                bottom = math.floor(int(state.get("next_pipe_bottom_y"))/20)
                dist = math.floor(int(state.get("next_pipe_dist_to_player"))/20)
                qval = q_val_far[player_y][top][bottom][dist][:]
            
#             qval = q_val_array[player_y][top][bottom][dist][:]
            qval_av_action[0] = qval[0]
            qval_av_action[1] = qval[1]
            print(qval[0]," et ", qval[1])
            action = (np.argmax(qval_av_action))
            print("action = ", action)
        #Take action, observe new state S'
        #Observe reward
        reward = p.act(119*action)
        state = game.getGameState()

            #Get max_Q(S',a)
        if (player_y - top < n_near_width/2 and player_y - bottom < n_near_width/2 and dist < n_near_dist):
            newQ = q_val_near[player_y - top + math.floor(n_near_width/2)][player_y - bottom + math.floor(n_near_width/2)][dist][:]
        else :
            player_y = math.floor(int(state.get("player_y"))/20) 
            top = math.floor(int(state.get("next_pipe_top_y"))/20)
            bottom = math.floor(int(state.get("next_pipe_bottom_y"))/20)
            dist = math.floor(int(state.get("next_pipe_dist_to_player"))/20)
            print([player_y, top , bottom, dist])
            newQ = q_val_far[player_y][top][bottom][dist][:]
        maxQ = np.max(newQ)       

        custom_reward = reward + c_reward(state)

        if reward != -5: #non-terminal state
            update = (custom_reward  + gamma * maxQ)
        else:
            update = reward
        newQ[action] += update #target output        
        
        print("Game #: %s" % (i,))

16
random action =  0
[12, 2, 7, 13]


TypeError: c_reward() missing 1 required positional argument: 'old_state'

In [8]:
n_dist = 100
n_next_dist = 100

n_vel = 20

n_width = math.floor(2*player_y_max/5) + 1 
n_next_width = 10

q_val_array = np.zeros((n_vel, n_width, n_dist, n_next_width, n_next_dist, 2))

epochs = 1000
game = FlappyBird()
p = PLE(game, frame_skip=1, num_steps=1, force_fps=True, display_screen=True)

p.init()

for i in range(epochs):
    p.init()
    old_state = game.getGameState()
    while(not p.game_over()):
        state = game.getGameState()
        
        if (state.get("player_y_vel") != None):
            vel = 10 + int(state.get("player_y_vel"))
        else:
            vel = 0
        
        player_y_rel = player_y_max + int(state.get("player_y")) - int(state.get("next_pipe_top_y"))
        dist = int(state.get("next_pipe_dist_to_player"))
        
        next_player_y_rel = player_y_max + int(state.get("player_y")) - int(state.get("next_next_pipe_top_y"))
        next_dist = int(state.get("next_next_pipe_dist_to_player"))       

        index_player_y_rel = math.floor(player_y_rel/5)
        index_dist = math.floor(dist / 4)
        
        index_next_player_y_rel = math.floor(next_player_y_rel/100)
        index_next_dist = math.floor(next_dist / 6)
        
        
        if (random.random() < epsilon(i)): # exploration exploitation strategy    
            action = int(bool(random.getrandbits(1)) == True)
            print("random action = ", action)
        else: #choose best action from Q(s,a) values

            qval = q_val_array[vel][index_player_y_rel][index_dist][index_next_player_y_rel][index_next_dist][:]
            qval_av_action = [-9999]*2
            qval_av_action[0] = qval[0]
            qval_av_action[1] = qval[1]
            print(qval[0]," et ", qval[1])
            action = (np.argmax(qval_av_action))

        #Take action, observe new state S'
        #Observe reward
        reward = p.act(119*action)
        
        state = game.getGameState()
        custom_reward = reward + c_reward(state, old_state)
        
        if (state.get("player_y_vel") != None):
            vel = 10 + int(state.get("player_y_vel"))
        else:
            vel = 0
            
        player_y_rel = player_y_max + int(state.get("player_y")) - int(state.get("next_pipe_top_y"))
        dist = int(state.get("next_pipe_dist_to_player"))
        
        next_player_y_rel = player_y_max + int(state.get("player_y")) - int(state.get("next_next_pipe_top_y"))
        next_dist = int(state.get("next_next_pipe_dist_to_player"))       

        index_player_y_rel = math.floor(player_y_rel/5)
        index_dist = math.floor(dist / 4)
        
        index_next_player_y_rel = math.floor(next_player_y_rel/100)
        index_next_dist = math.floor(next_dist / 6)

        #Get max_Q(S',a)
        newQ = q_val_array[vel][index_player_y_rel][index_dist][index_next_player_y_rel][index_next_dist][:]
        maxQ = np.max(newQ)       
        
        old_state = state
        if reward != -5: #non-terminal state
            update = (custom_reward  + gamma * maxQ)
        else:
            update = reward
        newQ[action] += update #target output        
        
        print("Game #: %s" % (i,))

random action =  0
-8.0
Game #: 0
random action =  1
0.0
Game #: 0
random action =  0
1.0
Game #: 0
random action =  0
2.0
Game #: 0
random action =  0
3.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  0
-7.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  1
0.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  1
0.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  0
-7.0
Game #: 0
random action =  0
-6.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  0
-8.0
Game #: 0
random action =  1
-8.0
Game #: 0
random action =  1
0

random action =  0
-5.0
Game #: 6
random action =  0
-4.0
Game #: 6
random action =  1
-8.0
Game #: 6
random action =  1
0.0
Game #: 6
random action =  1
-8.0
Game #: 6
random action =  1
0.0
Game #: 6
random action =  1
-8.0
Game #: 6
random action =  1
0.0
Game #: 6
random action =  0
1.0
Game #: 6
random action =  1
-8.0
Game #: 6
random action =  1
0.0
Game #: 6
random action =  0
1.0
Game #: 6
random action =  0
2.0
Game #: 6
random action =  0
3.0
Game #: 6
random action =  1
-8.0
Game #: 6
random action =  0
-8.0
Game #: 6
0.0  et  0.0
-7.0
Game #: 6
random action =  1
-8.0
Game #: 7
random action =  1
0.0
Game #: 7
random action =  1
-8.0
Game #: 7
0.0  et  0.1
0.0
Game #: 7
random action =  1
-8.0
Game #: 7
random action =  1
0.0
Game #: 7
random action =  0
1.0
Game #: 7
random action =  0
2.0
Game #: 7
random action =  0
3.0
Game #: 7
random action =  1
-8.0
Game #: 7
random action =  0
-8.0
Game #: 7
random action =  0
-7.0
Game #: 7
random action =  0
-6.0
Game #: 7
random

random action =  1
-8.0
Game #: 11
random action =  1
0.0
Game #: 11
random action =  0
1.0
Game #: 11
random action =  0
2.0
Game #: 11
0.1  et  0.0
3.0
Game #: 11
random action =  0
4.0
Game #: 11
random action =  0
5.0
Game #: 11
random action =  1
-8.0
Game #: 11
random action =  1
0.0
Game #: 11
random action =  1
-8.0
Game #: 11
random action =  1
0.0
Game #: 11
random action =  0
1.0
Game #: 11
random action =  0
2.0
Game #: 11
random action =  0
3.0
Game #: 11
random action =  0
4.0
Game #: 11
0.1  et  0.0
5.0
Game #: 11
0.1  et  0.0
6.0
Game #: 11
0.1  et  0.0
7.0
Game #: 11
random action =  1
-8.0
Game #: 11
random action =  0
-8.0
Game #: 11
random action =  1
-8.0
Game #: 11
random action =  1
0.0
Game #: 11
random action =  0
1.0
Game #: 11
random action =  1
-8.0
Game #: 11
random action =  0
-8.0
Game #: 11
random action =  0
-7.0
Game #: 11
random action =  1
-8.0
Game #: 11
random action =  0
-8.0
Game #: 11
0.0  et  0.0
-7.0
Game #: 11
random action =  0
-6.0
Game #: 

-8.0
Game #: 20
random action =  1
0.0
Game #: 20
random action =  1
-8.0
Game #: 20
0.0  et  0.0
-8.0
Game #: 20
random action =  0
-7.0
Game #: 20
0.0  et  0.0
-15.0
Game #: 21
random action =  0
-15.0
Game #: 21
random action =  0
-14.0
Game #: 21
0.1  et  0.0
-13.0
Game #: 21
random action =  0
-12.0
Game #: 21
random action =  0
-11.0
Game #: 21
random action =  0
-10.0
Game #: 21
random action =  1
-8.0
Game #: 21
random action =  1
0.0
Game #: 21
random action =  1
-8.0
Game #: 21
0.0  et  0.0
-8.0
Game #: 21
random action =  0
-7.0
Game #: 21
0.0  et  0.0
-6.0
Game #: 21
random action =  1
-8.0
Game #: 21
random action =  0
-8.0
Game #: 21
random action =  1
-8.0
Game #: 21
random action =  1
0.0
Game #: 21
random action =  0
1.0
Game #: 21
random action =  1
-8.0
Game #: 21
random action =  0
-8.0
Game #: 21
random action =  0
-7.0
Game #: 21
random action =  1
-8.0
Game #: 21
0.0  et  0.0
-8.0
Game #: 21
random action =  0
-7.0
Game #: 21
random action =  0
-6.0
Game #: 21
ra

random action =  1
-8.0
Game #: 30
random action =  0
-8.0
Game #: 30
random action =  0
-7.0
Game #: 30
random action =  1
-8.0
Game #: 30
random action =  0
-8.0
Game #: 30
random action =  1
-8.0
Game #: 30
random action =  1
0.0
Game #: 30
0.0  et  0.0
1.0
Game #: 30
random action =  0
2.0
Game #: 30
random action =  0
3.0
Game #: 30
random action =  1
-8.0
Game #: 30
0.0  et  0.1
0.0
Game #: 30
random action =  0
1.0
Game #: 30
random action =  0
2.0
Game #: 30
random action =  1
-8.0
Game #: 30
0.0  et  0.1
0.0
Game #: 30
random action =  1
-8.0
Game #: 30
0.0  et  0.0
-8.0
Game #: 30
random action =  1
-8.0
Game #: 30
random action =  1
0.0
Game #: 30
random action =  0
1.0
Game #: 30
random action =  0
2.0
Game #: 30
0.1  et  0.0
3.0
Game #: 30
0.1  et  0.0
4.0
Game #: 30
random action =  0
5.0
Game #: 30
random action =  1
-8.0
Game #: 30
0.0  et  0.0
-8.0
Game #: 30
0.0  et  0.0
-7.0
Game #: 30
random action =  0
-6.0
Game #: 30
0.0  et  0.0
-5.0
Game #: 30
random action =  1

...

0.0  et  0.0
-4.0
Game #: 670
0.0  et  0.0
-3.0
Game #: 670
0.0  et  0.0
-2.0
Game #: 670
0.0  et  0.0
-1.0
Game #: 670
0.0  et  0.0
0.0
Game #: 670
0.0  et  0.0
1.0
Game #: 670
0.1  et  0.0
2.0
Game #: 670
0.1  et  0.0
3.0
Game #: 670
random action =  1
-8.0
Game #: 670
0.0  et  0.0
-8.0
Game #: 670
0.0  et  0.0
-7.0
Game #: 670
random action =  0
-6.0
Game #: 670
0.0  et  0.0
-5.0
Game #: 670
0.0  et  0.0
-4.0
Game #: 670
random action =  0
-3.0
Game #: 670
0.0  et  0.0
-2.0
Game #: 670
0.199  et  0.0
-1.0
Game #: 670
0.0  et  0.0
0.0
Game #: 670
0.0  et  0.0
1.0
Game #: 670
random action =  1
-8.0
Game #: 670
0.0  et  0.0
-8.0
Game #: 670
0.0  et  0.0
-7.0
Game #: 670
0.0  et  0.0
-6.0
Game #: 670
0.0  et  0.0
-5.0
Game #: 670
0.0  et  0.0
-4.0
Game #: 670
0.0  et  0.0
-3.0
Game #: 670
0.0  et  0.0
-2.0
Game #: 670
0.0  et  0.0
-1.0
Game #: 670
0.0  et  0.0
0.0
Game #: 670
0.0  et  0.0
-8.0
Game #: 671
75.2405549  et  0.0
-8.0
Game #: 671
0.1  et  0.0
-7.0
Game #: 671
0.49601  et  0

0.49402  et  0.199
-2.0
Game #: 675
0.69501  et  0.099
-1.0
Game #: 675
0.8880599  et  0.0
0.0
Game #: 675
random action =  0
1.0
Game #: 675
0.199  et  0.099
2.0
Game #: 675
0.0  et  0.0
3.0
Game #: 675
0.0  et  0.0
4.0
Game #: 675
0.099  et  0.1
-8.0
Game #: 675
0.0  et  0.69501
0.0
Game #: 675
0.0  et  0.0
1.0
Game #: 675
2.344458402  et  0.299
2.0
Game #: 675
random action =  1
-8.0
Game #: 675
1.2781198  et  3.41203381798
0.0
Game #: 675
random action =  0
1.0
Game #: 675
random action =  1
-8.0
Game #: 675
0.7840998  et  0.976258802
0.0
Game #: 675
0.0  et  0.0
1.0
Game #: 675
0.39601  et  0.0
2.0
Game #: 675
0.39402  et  0.1
3.0
Game #: 675
1.1840699  et  0.0
4.0
Game #: 675
0.099  et  0.1
-8.0
Game #: 675
random action =  1
0.0
Game #: 675
0.1  et  2.163249201
-8.0
Game #: 675
0.199  et  0.29701
0.0
Game #: 675
0.0  et  0.59501
-8.0
Game #: 675
0.0  et  0.0
-8.0
Game #: 675
random action =  1
-8.0
Game #: 675
0.0  et  0.0
-8.0
Game #: 675
random action =  0
-7.0
Game #: 675
0.0

-8.0
Game #: 685
0.7880599  et  0.880179301
0.0
Game #: 685
1.560358602  et  3.4720624818
-8.0
Game #: 685
1.660358602  et  1.84375501598
0.0
Game #: 685
0.1  et  0.099
1.0
Game #: 685
10.3798226281  et  2.24756680899
2.0
Game #: 685
11.1137326154  et  2.04275501598
3.0
Game #: 685
3.38666606782  et  3.22087481598
4.0
Game #: 685
3.38567606782  et  1.84375501598
5.0
Game #: 685
random action =  0
6.0
Game #: 685
0.0  et  0.0
7.0
Game #: 685
0.29601  et  0.299
-8.0
Game #: 685
0.0  et  0.1
0.0
Game #: 685
0.39601  et  0.4920499
-8.0
Game #: 685
0.0  et  0.1
0.0
Game #: 685
0.7880599  et  0.780179301
1.0
Game #: 685
6.99844395988  et  0.0
2.0
Game #: 685
0.39601  et  0.0
3.0
Game #: 685
random action =  1
-8.0
Game #: 685
2.44357680899  et  3.7913702419
0.0
Game #: 685
0.39601  et  4.27365794989
-8.0
Game #: 685
0.199  et  0.29701
0.0
Game #: 685
0.1  et  0.099
1.0
Game #: 685
2.73075541498  et  0.7870898
2.0
Game #: 685
0.0  et  0.0
3.0
Game #: 685
random action =  0
4.0
Game #: 685
0.2

SystemExit: 

C:\Users\Pinchemel\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [69]:
def FlappyPolicy(state, screen):
    
    player_y = math.floor(int(state.get("player_y"))/5) 
    top = math.floor(int(state.get("next_pipe_top_y"))/5)
    bottom = math.floor(int(state.get("next_pipe_bottom_y"))/5)
    dist = int(state.get("next_pipe_dist_to_player"))
    
    qval = q_val_array[player_y][top][bottom][dist][:]
    qval_av_action[0] = qval[0]
    qval_av_action[1] = qval[1]
    print(qval[0]," et ", qval[1])
    return  119*(np.argmax(qval_av_action))

    
    
def greedy_action(network, state):
    Q = network.predict(np.array([x]))
    return np.argmax(Q)


def reward(state):
    player_y = float(state.get("player_y")) 
    dist = float(state.get("next_pipe_dist_to_player"))
    top = float(state.get("next_pipe_top_y"))
    bottom = float(state.get("next_pipe_bottom_y"))
    
    reward = abs(player_y - 0.5 * (top + bottom)) - 0.5 * (bottom - top)
#     print(reward)    
    if(player_y > top and player_y < bottom):
        return 1
    else:
        return 0

In [70]:
#import FlappyPolicy

game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=False, display_screen=True)

p.init()
reward = 0.0

nb_games = 100
cumulated = np.zeros((nb_games))

for i in range(nb_games):
    p.reset_game()
    
    while(not p.game_over()):
        state = game.getGameState()
        screen = p.getScreenRGB()
        action=FlappyPolicy(state, screen) ### Your job is to define this function.
        
        reward = p.act(action)
        cumulated[i] = cumulated[i] + reward

average_score = np.mean(cumulated)
max_score = np.max(cumulated)

0.0  et  0.0
5.12347188264  et  0.0
0.121026894866  et  0.0
0.116136919315  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
5.15217391304  et  0.0
0.149616368286  et  0.0
0.144501278772  et  0.0
0.136828644501  et  0.0
0.126598465473  et  0.0
0.113810741688  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  e

0.0665722379603  et  0.0
0.0382436260623  et  0.0
0.00991501416431  et  0.0
-0.0184135977337  et  0.0
0.0  et  0.0042492917847
0.0  et  0.0042492917847
0.0  et  0.0269121813031
0.0  et  0.0269121813031
0.0  et  0.0495750708215
0.0  et  0.0495750708215
0.0  et  0.0722379603399
0.0  et  0.0722379603399
0.0  et  0.0949008498584
0.0  et  0.0949008498584
0.0  et  0.117563739377
0.0  et  0.117563739377
0.0  et  0.140226628895
0.0  et  0.140226628895
0.0  et  0.162889518414
0.0  et  0.162889518414
0.0  et  0.185552407932
0.0  et  0.185552407932
0.0  et  0.20821529745
0.0  et  0.20821529745
0.0  et  0.230878186969
0.0  et  0.230878186969
0.0  et  0.253541076487
0.0  et  0.253541076487
0.0  et  0.276203966006
0.277464788732  et  0.550894106851
0.0  et  0.298866855524
0.0  et  0.298866855524
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0

0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0
0.0  et  0.0

SystemExit: 

C:\Users\Pinchemel\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
